In [1]:
import os
import glob
import shutil
import time

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import h2o
from h2o.automl import H2OAutoML

In [2]:
# Pull in CSVs
all_hexagons = pd.read_csv("all_hexagons_50c_grouped_withCounties.csv")
records = pd.read_csv("MOD_DF_0723.csv")

all_hexagons.set_index('subwatershed', inplace=True)

# Join data frames to make master file. Join key = 'subwatershed'
joined = records.join(all_hexagons, on='subwatershed')

In [3]:
# Drop 'bad data'
joined = joined[joined['subwatershed'] != 41900000200]
# Drop rows with blanks
joined = joined.dropna()

In [4]:
# Create directory
folder_name = time.strftime("%Y%m%d_%H.%M.%S")
try:
    os.mkdir('Runs/{}'.format(folder_name))
    out_directory = os.path.join('Runs', folder_name)
except OSError:
    print('Something went wrong')
    
# Save the joined data to csv
joined.to_csv(os.path.join(out_directory, 'joined_data.csv'))

In [5]:
joined.head()

,subwatershed,area,perimeter,circulatory_ratio,relief,avg_slope,watershed_length,elongation_ratio,drainage_density,shape_factor,...,flow_0_exceedence_prob,flow_0.1_exceedence_prob,flow_1_exceedence_prob,flow_10_exceedence_prob,flow_50_exceedence_prob,slope_of_flow_duration_curve,mean_policy,mean_claim,policy_total_building_coverage_avg,claims_total_building_insurance_coverage_avg
1,51202080201,83.362712,59.144456,0.299470,122.994843,5.590747,2.505820,4.111413,1.536133,0.075323,...,0.062106,0.036239,0.013721,0.003189,0.000567,-0.028288,84943.70200,5473.359624,3212.713978,1180.826665
2,51201110302,71.911106,43.130187,0.485784,53.171231,2.061335,4.160915,2.299663,1.186615,0.240759,...,0.037884,0.019586,0.006002,0.002000,0.000573,-0.016530,88263.86360,1236.275576,25613.870480,2713.363580
3,51201060507,42.800494,40.826927,0.322675,43.828071,1.045620,4.623489,1.596650,0.728583,0.499449,...,0.017746,0.008034,0.003114,0.001376,0.000681,-0.006407,94313.96496,13088.551960,27111.136390,28374.903350
4,71200010303,68.063067,51.205475,0.326203,31.397017,1.052509,5.454146,1.706806,0.575172,0.437061,...,0.002818,0.001514,0.001065,0.000428,0.000131,-0.000694,141562.36570,15387.941280,33976.722030,43.018237
5,50800030717,81.071752,51.841626,0.379072,133.800197,5.323677,2.995907,3.391263,1.273545,0.110710,...,0.027930,0.016684,0.003529,0.002116,0.000890,-0.016105,85883.67423,12641.896310,63568.611420,9695.319736


In [6]:
# Normalize the rows to 0-1, excluding the watershed identifier
scaler = MinMaxScaler(feature_range=(0, 1))
column_list = list(joined.columns)
column_list.remove('subwatershed')

joined[column_list] = scaler.fit_transform(joined[column_list])

In [7]:
joined.head()

,subwatershed,area,perimeter,circulatory_ratio,relief,avg_slope,watershed_length,elongation_ratio,drainage_density,shape_factor,...,flow_0_exceedence_prob,flow_0.1_exceedence_prob,flow_1_exceedence_prob,flow_10_exceedence_prob,flow_50_exceedence_prob,slope_of_flow_duration_curve,mean_policy,mean_claim,policy_total_building_coverage_avg,claims_total_building_insurance_coverage_avg
1,51202080201,0.543402,0.366385,0.370504,0.560811,0.392208,0.055040,0.827977,0.503853,0.001518,...,0.118630,0.106424,0.123588,0.189331,0.244287,0.895287,0.285431,0.092687,0.023990,0.007350
2,51201110302,0.445450,0.178116,0.687820,0.214932,0.124870,0.161939,0.435096,0.360935,0.012732,...,0.072362,0.057519,0.054066,0.118750,0.246878,0.938812,0.296588,0.020935,0.191268,0.016889
3,51201060507,0.196450,0.151038,0.410024,0.168650,0.047933,0.191816,0.282645,0.173645,0.030266,...,0.033897,0.023595,0.028049,0.081692,0.293509,0.976284,0.316917,0.221644,0.202448,0.176615
4,71200010303,0.412535,0.273052,0.416034,0.107071,0.048455,0.245466,0.306533,0.110915,0.026037,...,0.005382,0.004445,0.009592,0.025415,0.056236,0.997430,0.475683,0.260583,0.253716,0.000268
5,50800030717,0.523806,0.280530,0.506076,0.614337,0.371979,0.086694,0.671811,0.396481,0.003917,...,0.053349,0.048997,0.031785,0.125634,0.383398,0.940384,0.288590,0.214081,0.474689,0.060347


In [8]:
# Should return claims field
column_list[72]

'claims_total_building_insurance_coverage_avg'

In [9]:
scaler_by = scaler.scale_[72]
scaler_min = scaler.min_[72]

print("Note: {} values were scaled by multiplying by {:.12f} and adding {:.10f}".format(
    column_list[72],
    scaler_by,
    scaler_min)
)

Note: claims_total_building_insurance_coverage_avg values were scaled by multiplying by 0.000006224341 and adding 0.0000000000


In [10]:
joined.to_csv(os.path.join(out_directory, 'final_data_scaled.csv'))

In [11]:
# Use 70% of the data for training, 15% for testing, 15% for validation.
# Subset data for training, testing, validation
training_df = joined.sample(frac=.7, random_state=663168)
testing_df = joined.loc[~joined.index.isin(training_df.index)]
validation_df = testing_df.sample(frac=.5, random_state=663168)
testing_df = testing_df.loc[~testing_df.index.isin(validation_df.index)]

In [12]:
training_df.to_csv(os.path.join(out_directory, 'training_data.csv'))
testing_df.to_csv(os.path.join(out_directory, 'testing_data.csv'))
validation_df.to_csv(os.path.join(out_directory, 'validation_data.csv'))

In [14]:
h2o.init()

# Upload training data:
train = h2o.upload_file(
    os.path.join(out_directory, 'training_data.csv'),
    header=1,
    sep=',',
    skipped_columns=[0, 1]
)

# Upload testing data:
test = h2o.upload_file(
    os.path.join(out_directory, 'testing_data.csv'),
    header=1,
    sep=',',
    skipped_columns=[0, 1]
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 11.0.6+8-b765.1, mixed mode)
  Starting server from C:\Users\Ebarnes\AppData\Local\Continuum\anaconda3\envs\h2o_ml\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Ebarnes\AppData\Local\Temp\tmp3cy90d7j
  JVM stdout: C:\Users\Ebarnes\AppData\Local\Temp\tmp3cy90d7j\h2o_ebarnes_started_from_python.out
  JVM stderr: C:\Users\Ebarnes\AppData\Local\Temp\tmp3cy90d7j\h2o_ebarnes_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.7
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_ebarnes_rtjydn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.928 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
x = train.columns
y = 'claims_total_building_insurance_coverage_avg'
x.remove(y)

In [16]:
aml = H2OAutoML(max_models=40, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
14:09:24.375: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [17]:
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200727_140924,0.00984617,0.0992279,0.00984617,0.0711376,0.0829993
GBM_grid__1_AutoML_20200727_140924_model_1,0.0100165,0.100082,0.0100165,0.071731,0.0836456
StackedEnsemble_AllModels_AutoML_20200727_140924,0.0102315,0.101151,0.0102315,0.0721514,0.0839558
GBM_grid__1_AutoML_20200727_140924_model_8,0.0104941,0.102441,0.0104941,0.0732938,0.0851205
GBM_grid__1_AutoML_20200727_140924_model_12,0.0106982,0.103432,0.0106982,0.07287,0.0858451
GBM_2_AutoML_20200727_140924,0.0107373,0.103621,0.0107373,0.0734397,0.0862715
GBM_4_AutoML_20200727_140924,0.0107615,0.103737,0.0107615,0.071284,0.0860481
GBM_3_AutoML_20200727_140924,0.0107667,0.103763,0.0107667,0.0729952,0.0863983
GBM_grid__1_AutoML_20200727_140924_model_7,0.0108209,0.104024,0.0108209,0.0736476,0.0869181
GBM_grid__1_AutoML_20200727_140924_model_9,0.0108614,0.104218,0.0108614,0.0731874,0.0865994


In [18]:
perf = aml.leader.model_performance(test)
print(perf)


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.017622400250602036
RMSE: 0.13274938888974983
MAE: 0.08164309687306809
RMSLE: 0.10319773643499221
R^2: 0.16365381075627872
Mean Residual Deviance: 0.017622400250602036
Null degrees of freedom: 118
Residual degrees of freedom: 116
Null deviance: 2.5097741007848713
Residual deviance: 2.097065629821642
AIC: -134.8841780136675

